In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

c:\Users\vivek\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### IOU

In [2]:
def calculate_iou(true_mask, predicted_mask):
    # Convert true mask to gray scale
    true_mask_gray = cv2.cvtColor(true_mask, cv2.COLOR_BGR2GRAY)
    # Convert predicted mask to gray scale
    predicted_mask_gray = cv2.cvtColor(predicted_mask, cv2.COLOR_BGR2GRAY)
    
    intersection = np.logical_and(true_mask_gray, predicted_mask_gray)
    union = np.logical_or(true_mask_gray, predicted_mask_gray)

    iou = np.sum(intersection) / np.sum(union)
    return iou

### Dice

In [3]:
def calculate_dice_score(true_mask, predicted_mask):
    # Ensure the masks are binary
    true_mask_binary = true_mask > 0
    predicted_mask_binary = predicted_mask > 0
    
    intersection = np.logical_and(true_mask_binary, predicted_mask_binary)

    dice_score = 0

    # Calculate Dice score only if both true and predicted masks are present
    if np.sum(true_mask_binary) > 0 and np.sum(predicted_mask_binary) > 0:
        # Calculate intersection area
        intersection_area = np.sum(intersection)
        # Calculate area of true mask
        true_area = np.sum(true_mask_binary)
        # Calculate area of predicted mask
        predicted_area = np.sum(predicted_mask_binary)
        
        # Calculate Dice score
        dice_score = 2 * intersection_area / (true_area + predicted_area)
    else:
        dice_score = 0

    return dice_score

### pixel wise accuracy

In [4]:
#pixel wise accuracy

def calculate_pixel_wise_accuracy(true_mask, predicted_mask):
    
    #convert true mask to gray scale
    true_mask_gray=cv2.cvtColor(true_mask,cv2.COLOR_BGR2GRAY)
    #convert predicted mask to gray scale
    predicted_mask_gray=cv2.cvtColor(predicted_mask,cv2.COLOR_BGR2GRAY)
    #calculate pixel wise accuracy
    #number of pixels in true mask
    true_positive=np.sum((true_mask_gray==255) & (predicted_mask_gray==255))
    false_positive=np.sum((true_mask_gray==0) & (predicted_mask_gray==255))
    false_negative=np.sum((true_mask_gray==255) & (predicted_mask_gray==0))
    true_negative=np.sum((true_mask_gray==0) & (predicted_mask_gray==0))


    #pixel wise accuracy
    pixel_wise_accuracy=(true_positive+true_negative)/(true_positive+true_negative+false_positive+false_negative)

    return pixel_wise_accuracy

### Bbox

In [5]:
#
benign_predictions_dir=os.path.join('..', 'BreastSAM impl', 'SAM_predictions_BBox', 'benign')
benign_gt_dir=os.path.join('..','Dataset_BUSI_with_GT','benign')

#file names in benign_gt_dir =benign (1)_mask.png
#file names in benign_predictions_dir =benign (1)_mask_prediction.png


list_of_benign_predictions_dir=os.listdir(benign_predictions_dir)

df=pd.DataFrame(columns=['File','IOU','Dice Score','Pixel Wise Accuracy'])

for files in list_of_benign_predictions_dir:
    
    #read true mask
    true_mask=cv2.imread(os.path.join(benign_gt_dir,files.replace('_mask_prediction.png','_mask.png')))
    #read predicted mask
    predicted_mask=cv2.imread(os.path.join(benign_predictions_dir,files))
    
    #calculate iou
    iou=calculate_iou(true_mask,predicted_mask)
    #calculate dice score
    dice_score=calculate_dice_score(true_mask,predicted_mask)
    #calculate pixel wise accuracy
    pixel_wise_accuracy=calculate_pixel_wise_accuracy(true_mask,predicted_mask)

    #dont use append method to add rows to dataframe

    df.loc[len(df)]=[files,iou,dice_score,pixel_wise_accuracy]
    


#save the dataframe to a csv file
df.to_csv(os.path.join('..','BreastSAM impl','benign_metrics.csv'),index=False)

In [7]:
def calculate_metrics(benign_gt_dir, benign_predictions_dir,csv_file_name):
    list_of_benign_predictions_dir = os.listdir(benign_predictions_dir)

    df = pd.DataFrame(columns=['File', 'IOU', 'Dice Score', 'Pixel Wise Accuracy'])

    for files in list_of_benign_predictions_dir:
        # read true mask
        #print(files)
        true_mask = cv2.imread(os.path.join(benign_gt_dir, files.replace('_mask_prediction.png', '_mask.png')))
        # read predicted mask
        predicted_mask = cv2.imread(os.path.join(benign_predictions_dir, files))

        # calculate iou
        iou = calculate_iou(true_mask, predicted_mask)
        # calculate dice score
        dice_score = calculate_dice_score(true_mask, predicted_mask)
        # calculate pixel wise accuracy
        pixel_wise_accuracy = calculate_pixel_wise_accuracy(true_mask, predicted_mask)

        df.loc[len(df)] = [files, iou, dice_score, pixel_wise_accuracy]


    # save the dataframe to a csv file
    df.to_csv(csv_file_name, index=False)
    
    print('Metrics saved to ',csv_file_name)
    average_dice_score=np.mean(df['Dice Score'])
    average_iou=np.mean(df['IOU'])
    average_pixel_wise_accuracy=np.mean(df['Pixel Wise Accuracy'])
    
    return average_dice_score,average_iou,average_pixel_wise_accuracy

## only box prompt

In [9]:
benign_predictions_dir=os.path.join('..', 'BreastSAM impl', 'SAM_predictions_BBox', 'benign')
benign_gt_dir=os.path.join('..','Dataset_BUSI_with_GT','benign')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(benign_gt_dir,benign_predictions_dir,os.path.join('..','BreastSAM impl','benign_box_metrics.csv'))

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

Metrics saved to  ..\BreastSAM impl\benign_box_metrics.csv
Average Dice Score: 0.8639273011884666
Average IOU: 0.7751496027403703
Average Pixel Wise Accuracy: 0.9836931349911328


In [103]:
malignant_predictions_dir=os.path.join('..', 'BreastSAM impl', 'SAM_predictions_BBox', 'malignant')
malignant_gt_dir=os.path.join('..','Dataset_BUSI_with_GT','malignant')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(malignant_gt_dir,malignant_predictions_dir,os.path.join('..','BreastSAM impl','malignant_box_metrics.csv'))

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

                                  File       IOU  Dice Score  \
0    malignant (1)_mask_prediction.png  0.768956    0.869390   
1   malignant (10)_mask_prediction.png  0.690886    0.817188   
2  malignant (100)_mask_prediction.png  0.693924    0.819310   
3  malignant (101)_mask_prediction.png  0.633904    0.775938   
4  malignant (102)_mask_prediction.png  0.579419    0.733712   

   Pixel Wise Accuracy  
0             0.918868  
1             0.989222  
2             0.981323  
3             0.948875  
4             0.946530  
Average Dice Score: 0.8231403836728571
Average IOU: 0.7083671832802122
Average Pixel Wise Accuracy: 0.954822882845514


## with box and point prompt

In [104]:
benign_gt_dir=os.path.join('..','Dataset_BUSI_with_GT','benign')
benign_point_box_dir=os.path.join('..','BreastSAM impl','SAM_predictions_BBoxPoint','benign')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(benign_gt_dir,benign_point_box_dir,os.path.join('..','BreastSAM impl','benign_point_box_metrics.csv'))

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

                               File       IOU  Dice Score  Pixel Wise Accuracy
0    benign (1)_mask_prediction.png  0.838825    0.912349             0.999316
1   benign (10)_mask_prediction.png  0.941567    0.969904             0.995155
2  benign (100)_mask_prediction.png  0.826915    0.905258             0.991851
3  benign (101)_mask_prediction.png  0.866155    0.928278             0.997905
4  benign (102)_mask_prediction.png  0.925291    0.961196             0.996302
Average Dice Score: 0.8771654120817105
Average IOU: 0.79010434918551
Average Pixel Wise Accuracy: 0.9844730317006107


In [105]:
malignant_gt_dir=os.path.join('..','Dataset_BUSI_with_GT','malignant')
malignant_point_box_dir=os.path.join('..','BreastSAM impl','SAM_predictions_BBoxPoint','malignant')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(malignant_gt_dir,malignant_point_box_dir,os.path.join('..','BreastSAM impl','malignant_point_box_metrics.csv'))

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)


                                  File       IOU  Dice Score  \
0    malignant (1)_mask_prediction.png  0.754960    0.860373   
1   malignant (10)_mask_prediction.png  0.664978    0.798783   
2  malignant (100)_mask_prediction.png  0.616424    0.762701   
3  malignant (101)_mask_prediction.png  0.630228    0.773178   
4  malignant (102)_mask_prediction.png  0.576915    0.731701   

   Pixel Wise Accuracy  
0             0.913572  
1             0.988709  
2             0.976763  
3             0.948377  
4             0.946221  
Average Dice Score: 0.8304638058228664
Average IOU: 0.7155563276327351
Average Pixel Wise Accuracy: 0.9551953264603636


### Point 

In [106]:
benign_gt_dir=os.path.join('..','Dataset_BUSI_with_GT','benign')
benign_point_box_dir=os.path.join('..','BreastSAM impl','SAM_predictions_Point','benign')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(benign_gt_dir,benign_point_box_dir,os.path.join('..','BreastSAM impl','benign_point_metrics.csv'))

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

                               File       IOU  Dice Score  Pixel Wise Accuracy
0    benign (1)_mask_prediction.png  0.846985    0.917154             0.999358
1   benign (10)_mask_prediction.png  0.943227    0.970784             0.995287
2  benign (100)_mask_prediction.png  0.822899    0.902847             0.991668
3  benign (101)_mask_prediction.png  0.872422    0.931865             0.998002
4  benign (102)_mask_prediction.png  0.920207    0.958446             0.995979
Average Dice Score: 0.6771114339553682
Average IOU: 0.5896810091898083
Average Pixel Wise Accuracy: 0.896623810514078


In [107]:
malignant_gt_dir=os.path.join('..','Dataset_BUSI_with_GT','malignant')
malignant_point_box_dir=os.path.join('..','BreastSAM impl','SAM_predictions_Point','malignant')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(malignant_gt_dir,malignant_point_box_dir,os.path.join('..','BreastSAM impl','malignant_point_metrics.csv'))

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)


                                  File       IOU  Dice Score  \
0    malignant (1)_mask_prediction.png  0.675349    0.806219   
1   malignant (10)_mask_prediction.png  0.088316    0.162298   
2  malignant (100)_mask_prediction.png  0.355373    0.524392   
3  malignant (101)_mask_prediction.png  0.604435    0.753455   
4  malignant (102)_mask_prediction.png  0.513201    0.678299   

   Pixel Wise Accuracy  
0             0.889517  
1             0.972539  
2             0.961133  
3             0.944888  
4             0.938189  
Average Dice Score: 0.5883291313479885
Average IOU: 0.46138997781788166
Average Pixel Wise Accuracy: 0.8071209403626128
